In [4]:
!pip install googletrans
!pip install rapidfuzz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.2 MB/s eta 0:00:0000:0100:01


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
from googletrans import Translator
from rapidfuzz import process
import pandas as pd


In [6]:
# Bước 1: Đọc file CSV từ Kaggle
# Đảm bảo rằng bạn đã tải file CSV từ Kaggle và upload lên Google Colab
# Hoặc sử dụng Kaggle API để tải trực tiếp
file_path = "/kaggle/input/diseases-and-symptoms-dataset/Final_Augmented_dataset_Diseases_and_Symptoms.csv"  # Thay đổi đường dẫn tùy theo vị trí file của bạn
df = pd.read_csv(file_path)
# print(df.head())

In [7]:
# Lọc ra các triệu chứng (tức là tất cả các cột trừ cột "diseases")
symptoms = list(df.columns[df.columns != "diseases"])

# print("Các triệu chứng trong tập dữ liệu:")
# for symptom in symptoms:
#     print("-", symptom)

print("Độ dài triệu chứng : ", len(symptoms))

Độ dài triệu chứng :  377


In [8]:
# Giả sử df là DataFrame đã đọc từ file hoặc tạo sẵn
disease_list = df["diseases"].unique()

# print("Các bệnh có trong tập dữ liệu:")
# for disease in disease_list:
#     print("-", disease)

print("Độ dài của danh sách bệnh : ", len(disease_list))

Độ dài của danh sách bệnh :  773


In [ ]:
# # Triệu chứng tiếng Việt người dùng nhập
# user_symptoms_vi = [ "lo âu và căng thẳng thần kinh",
#     "khó thở",
#     "mất ngủ",
#     "đau nhói ngực"]

# # Dịch và khớp
# translator = Translator()
# matched_symptoms = []

# for symptom_vi in user_symptoms_vi:
#     translated = translator.translate(symptom_vi, src='vi', dest='en').text
#     match, score, _ = process.extractOne(translated, symptoms)
#     print(translated, "match : ", match, "score : ",score)
#     if score > 60:  # Ngưỡng độ tương đồng, có thể chỉnh
#         matched_symptoms.append(match)

# print("Triệu chứng khớp với mô hình:", matched_symptoms)

from googletrans import Translator
import pandas as pd

# Danh sách người dùng nhập bằng tiếng Việt
input_vi = [ "lo âu và căng thẳng thần kinh",
    "khó thở",
    "mất ngủ",
    "đau nhói ngực"]

# Khởi tạo translator
translator = Translator()

# Hàm kiểm tra khớp từ khóa
def match_keywords(translated_symptom, symptoms):
    for symptom in symptoms:
        if translated_symptom in symptom or symptom in translated_symptom:
            return symptom
    return None

# Dịch và kiểm tra
matched_symptoms = []
for symptom_vi in input_vi:
    translated = translator.translate(symptom_vi, src='vi', dest='en').text.lower()
    print(f"'{symptom_vi}' dịch sang: '{translated}'")
    
    matched = match_keywords(translated, symptoms)
    if matched:
        matched_symptoms.append(matched)

print("\n✅ Các triệu chứng khớp với mô hình:", matched_symptoms)



In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from googletrans import Translator

# Model dùng cho việc chuyển câu thành embedding
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Khởi tạo translator
translator = Translator()

# Dữ liệu người dùng nhập bằng tiếng Việt
user_input_vi = "Tôi cảm thấy khó thở và mệt mỏi"

# Dịch người dùng nhập từ tiếng Việt sang tiếng Anh (giả sử dịch chính xác)
# Bạn có thể thay thế bằng thư viện dịch hoặc bảng từ điển dịch
# user_input_en = "I feel shortness of breath and fatigue"
user_input_en = "Over the past few days, I've been experiencing persistent shortness of breath, even during minimal exertion, accompanied by a constant feeling of fatigue that doesn’t improve with rest. In addition, I've noticed occasional chest tightness, mild dizziness when standing up, and a general sense of weakness throughout the day. There are moments when I feel lightheaded, and I've also had difficulty concentrating or staying alert."
# user_input_en = translator.translate(user_input_vi, src='vi', dest='en').text.lower()
print(user_input_en)


# Tạo embedding
user_input_embedding = model.encode([user_input_en])
symptoms_embeddings = model.encode(symptoms)

# Tính độ tương đồng cosine
similarities = cosine_similarity(user_input_embedding, symptoms_embeddings)[0]

# Ghép triệu chứng và điểm tương đồng thành danh sách
symptom_scores = list(zip(symptoms, similarities))

# Lọc những triệu chứng có độ tương đồng > 60%
filtered_symptoms = [(s, score) for s, score in symptom_scores if score > 0.6]

# Sắp xếp theo độ tương đồng giảm dần
filtered_symptoms.sort(key=lambda x: x[1], reverse=True)

# In kết quả
if filtered_symptoms:
    print("Các triệu chứng có độ tương đồng > 60%:")
    for symptom, score in filtered_symptoms:
        print(f"- {symptom}: {score * 100:.2f}%")
else:
    print("Không có triệu chứng nào có độ tương đồng > 60%")

In [9]:
# Bước 2: Chuẩn bị dữ liệu
# Tách features (triệu chứng) và labels (bệnh)
X = df.drop(columns=["diseases"])  # Features: tất cả các cột trừ "diseases"
y = df["diseases"]  # Labels: cột "diseases"
# print(X.head())
# print(y.head())

In [10]:
# Bước 3: Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Bước 4: Huấn luyện mô hình Random Forest
model = RandomForestClassifier(random_state=42, n_estimators=100, warm_start=True)

# Huấn luyện từng cây và hiển thị tiến trình
for i in tqdm(range(20)):  # Thay 10 bằng số lượng cây (n_estimators)
    model.n_estimators = i + 1
    model.fit(X_train, y_train)

100%|██████████| 20/20 [00:49<00:00,  2.46s/it]


In [ ]:
# Bước 5: Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

In [12]:
import joblib
model_filename = 'disease_classify.joblib'
joblib.dump(model, model_filename)

['disease_classify.joblib']

In [13]:
# Bước 6: Đánh giá mô hình
print("Độ chính xác (Accuracy):", accuracy_score(y_test, y_pred))
print("Báo cáo phân loại (Classification Report):\n", classification_report(y_test, y_pred))

NameError: name 'y_pred' is not defined

In [14]:
# Bước 7: Dự đoán bệnh từ triệu chứng mới
# Danh sách các triệu chứng mới
new_symptoms = ["anxiety and nervousness", "shortness of breath", "sharp chest pain", "insomnia", "palpitations"]

# Tạo một dictionary với tất cả các triệu chứng trong X, giá trị mặc định là 0
new_symptoms_dict = {symptom: [0] for symptom in X.columns}

# Điền giá trị 1 cho các triệu chứng có trong new_symptoms
for symptom in new_symptoms:
    if symptom in new_symptoms_dict:
        new_symptoms_dict[symptom] = [1]

# Tạo DataFrame từ dictionary
new_df = pd.DataFrame(new_symptoms_dict)

# Dự đoán xác suất của các bệnh
predicted_proba = model.predict_proba(new_df)

# Lấy danh sách các bệnh (labels)
disease_labels = model.classes_

# Tạo DataFrame chứa xác suất của các bệnh
proba_df = pd.DataFrame(predicted_proba, columns=disease_labels)

# Lấy top 10 bệnh có xác suất cao nhất
top_10_diseases = proba_df.iloc[0].sort_values(ascending=False).head(10)

# Hiển thị kết quả
print("Top 10 bệnh dự đoán và xác suất tương ứng:")
for disease, proba in top_10_diseases.items():
    print(f"{disease}: {proba * 100:.2f}%")

Top 10 bệnh dự đoán và xác suất tương ứng:
anxiety: 80.00%
acute stress reaction: 10.00%
panic disorder: 5.00%
hypertensive heart disease: 5.00%
oral leukoplakia: 0.00%
oral mucosal lesion: 0.00%
oral thrush (yeast infection): 0.00%
orbital cellulitis: 0.00%
orthostatic hypotension: 0.00%
osteoarthritis: 0.00%
